In [1]:
from OldHelper import channelcollisions as old_cc
from Helper import load_coords
import numpy as np
import time

rand_input = np.random.random((50,6))

old_output = []

def old_test(i):
    s = time.time()
    old_output.append(old_cc(rand_input[i], load_coords("."))[2])
    return time.time() - s

#print("took", old_test(1), "s")

for i in range(50):
    old_output.append(old_cc(rand_input[i], load_coords("."))[2])

In [2]:
from Helper import channelcollisions as new_cc
import numpy as np
import time

new_output = []

def new_test(i):
    
    old_output = []

    s = time.time()
    new_output.append(new_cc(rand_input[i], load_coords("."))[2])
    return time.time() - s

#print("took", new_test(1), "s")

for i in range(50):
    new_output.append(new_cc(rand_input[i], load_coords("."))[2])

In [3]:
old_output[0] == new_output[0]

array([ True,  True,  True, False,  True,  True,  True, False,  True,
       False])

In [42]:
def check():
    rand_input = np.random.random((50,6))
    
    new_output = []
    old_output = []
    for i in range(50):
        old_output.append(old_cc(rand_input[i], load_coords("."))[2])
        new_output.append(new_cc(rand_input[i], load_coords("."))[2])
    
    for i in range(50):
        if not (np.round(old_output[i],8) == np.round(new_output[i],8)).all():
            return False
    return True

check()

True

In [8]:
import numpy as np
import numpy.ctypeslib as ctl
import ctypes
import time

def channelcollisions(line, file=None):
    
    libname = 'libcoll.so'
    libdir = './'
    lib=ctl.load_library(libname, libdir)

    c_double_p = ctypes.POINTER(ctypes.c_double)
    c_int_p = ctypes.POINTER(ctypes.c_int)

    channelcollisions = lib.channelcollisions
    channelcollisions.argtypes = [ctl.ndpointer(np.float64, 
                                             flags='aligned, c_contiguous'),
                                 c_int_p,
                                 c_int_p,
                                 c_double_p]

    #channelcollisions(np.array([1,2,3,4,5,6], dtype=np.float64))
    
    

    data1 = (ctypes.c_int * 30)()
    data2 = (ctypes.c_int * 30)()
    data3 = (ctypes.c_double * 30)()
    
    print("checkpoint 1: ", time.time() - start)

    res = channelcollisions(line.astype(np.float64), 
                            ctypes.cast(data1, c_int_p),
                            ctypes.cast(data2, c_int_p), 
                            ctypes.cast(data3, c_double_p))
    
    print("checkpoint 2: ", time.time() - start)
    
    data1 = np.array(data1)
    data2 = np.array(data2)
    data3 = np.array(data3)
    
    hit_channels = data1[data1!=0]
    miss_channels = data2[data2!=0]
    track_lengths = data3[data3!=0]
    
    return (list(hit_channels), list(miss_channels), track_lengths)

start = time.time()
channelcollisions(rand_input[0])
print("checkpoint 3: ", time.time() - start)

checkpoint 1:  0.015041112899780273
checkpoint 2:  0.028452157974243164
checkpoint 3:  0.02970123291015625


In [ ]:
new_cc(rand_input[0], load_coords('.'))

In [ ]:
old_cc(rand_input[0], load_coords('.'))

In [ ]:
rand_input[0]

In [ ]:
old_output

In [ ]:
new_output

In [ ]:
new_test(1)

In [ ]:
import matplotlib.pyplot as plt

[new_test(i) for i in range(50)] # calibrate


new_tests = [new_test(i) for i in range(50)]
old_tests = [old_test(i) for i in range(50)]



plt.hist(old_tests, np.linspace(0,.1,30), histtype='step', label='py')
plt.hist(new_tests, np.linspace(0,.1,30), histtype='step', label='pyx')
plt.legend()

In [ ]:
new_tests

In [ ]:
%%cython

import numpy as np
from scipy.interpolate import CubicSpline
from Helper import load_coords
import time

def lineplanecollision(planeNormal, planePoint, rayDirection, rayPoint, epsilon=1e-6):

    ndotu = planeNormal.dot(rayDirection)
    if abs(ndotu) < epsilon:
        return None

    t = -planeNormal.dot(rayPoint - planePoint) / ndotu

    return rayPoint + t * rayDirection


def linecubecollision(cubeCenter, cubeLength, rayDirection, rayPoint, epsilon=1e-6):

    cubeCollisions = []

    halfLength = cubeLength / 2.0

    directions = np.array([
        [0,0,halfLength], #up
        [0,halfLength,0], #front
        [halfLength,0,0], #right
    ])

    planeCollisions = []
    for i in range(6):
        if i >= 3:
            faceNormal = -directions[i%3] # to get down, back, left
        else:
            faceNormal = directions[i]

        facePoint = cubeCenter + faceNormal

        collision = lineplanecollision(faceNormal, facePoint, rayDirection, rayPoint)
        if collision is not None:
            planeCollisions.append(collision)

    #check if intersection is outside cube
    for collision in planeCollisions:

        inside = True
        for i in range(3):
            if collision[i] > (cubeCenter[i] + halfLength + epsilon) or collision[i] < (cubeCenter[i] - halfLength - epsilon):
                inside = False

        if inside:
            cubeCollisions.append(collision)

    return cubeCollisions


def channelcollisions(line, coords, epsilon=1e-6):

    cdef float[:] rayDirection = line[3:]
    cdef float[:] rayPoint = line[:3]

    #rayDirection = linepoints[1] - linepoints[0]
    #rayPoint = linepoints[0]
    cdef int cubeLength = 50

    #start = time.time()

    cdef int hit_channels[20]
    cdef int miss_channels[20]
    cdef float track_distances[20]
     
    # list of coords corresponding to channels
    # index corresponds to channel num - 1
    cdef float [:,:] coord_list = coords
    cdef int num_channels = len(coord_list)
    
    
    cdef int channel
    cdef float [:] cubeCenter
    cdef float [:] CP
    cdef float distance_to_line

    for i in range(num_channels):
        channel = i+1
        cubeCenter = coord_list[channel]

        #check if cubeCenter is within range of line
        
        CP = cubeCenter - rayPoint
        
        distance_to_line = np.abs(np.linalg.norm(cross(CP,rayDirection)) / np.linalg.norm(rayDirection))
        
        print(distance_to_line)

        #print(distance_to_line)

        if distance_to_line < cubeLength/2*np.sqrt(3) + epsilon:
        #if distance_to_line < cubeLength*np.sqrt(3) + epsilon:

            collision = linecubecollision(cubeCenter, cubeLength, rayDirection, rayPoint)
            if len(collision) == 2:
                hit_channels.append(channel)
                track_distances.append(np.linalg.norm(collision[1] - collision[0]))
            else:
                miss_channels.append(channel)

    return (hit_channels, miss_channels, track_distances)

# manually do crossproduct to avoid numpy overhead for small vectors
def cross(a, b):
    c = [a[1]*b[2] - a[2]*b[1], a[2]*b[0] - a[0]*b[2], a[0]*b[1] - a[1]*b[0]]
    return c

s = time.time()
channelcollisions(np.random.random(6), np.array(list(load_coords(".").values()),dtype=np.float32))
print(time.time() - s)

In [ ]:
%%cython

import numpy as np
from scipy.interpolate import CubicSpline
from Helper import load_coords
import time

def lineplanecollision(planeNormal, planePoint, rayDirection, rayPoint, epsilon=1e-6):

    ndotu = planeNormal.dot(rayDirection)
    if abs(ndotu) < epsilon:
        return None

    t = -planeNormal.dot(rayPoint - planePoint) / ndotu

    return rayPoint + t * rayDirection

cdef vector[float] a = [0,0,1]
cdef vector[float] b = [0,0,0]
cdef vector[float] c = [0,0,1]
cdef vector[float] d = [0,0,0]

lineplanecollision(a,b,c,d)

In [ ]:
from Helper import linecubecollision

linecubecollision(np.array([0,1,0]), 50, np.array([-2, 1.1, 1]), np.array([1,0,0]))


In [ ]:
[{i[0], i[1], i[2]} for i in load_coords('.').values()]

In [ ]:
import numpy as np
a = np.array([1,2,3])
b = np.array([4,5,6])

np.linalg.norm(a) / np.linalg.norm(b)

In [ ]:
from Helper import *
import time
coords = load_coords('.')

s = time.time()
for i in range(50):
    res = channelcollisions(np.array([1,2,3,4,5,6]), coords)
print(time.time() - s)

In [ ]:
res

In [ ]:
coords

In [ ]:
# import the module 
import ctypes
  
# load the library 
lib = ctypes.cdll.LoadLibrary('./libcoll.so') 
  
# create a Geek class 
class Collision(object): 
  
    # constructor 
    def __init__(self): 
  
        # attribute 
        self.obj = lib.Collision_new() 
  
    # define method 
    def channelcollisions(self, track): 
        lib.Collision_channelcollisions(self.obj, track) 
  
# create a Geek class object 
c = Collision() 
#c.channelcollisions.restype = ctypes.POINTER(ctypes.c_double * 10)
  
# # object method calling 
# c.channelcollisions()

In [ ]:
import numpy.ctypeslib as ctl
import ctypes
import numpy as np

libname = 'libcoll.so'
libdir = './'
lib=ctl.load_library(libname, libdir)

c_double_p = ctypes.POINTER(ctypes.c_double)
c_int_p = ctypes.POINTER(ctypes.c_int)

channelcollisions = lib.channelcollisions
channelcollisions.argtypes = [ctl.ndpointer(np.float64, 
                                         flags='aligned, c_contiguous'),
                             c_int_p,
                             c_int_p,
                             c_double_p]

#channelcollisions(np.array([1,2,3,4,5,6], dtype=np.float64))

data1 = (ctypes.c_int * 30)()
data2 = (ctypes.c_int * 30)()
data3 = (ctypes.c_double * 30)()

res = channelcollisions(np.array([1,2,3,4,5,6], dtype=np.float64), 
                        ctypes.cast(data1, c_int_p),
                        ctypes.cast(data2, c_int_p), 
                        ctypes.cast(data3, c_double_p))

data1 = np.array(data1)
data2 = np.array(data2)
data3 = np.array(data3)

hit_channels = data1[data1!=0]
missed_channels = data2[data2!=0]
track_lengths = data3[data3!=0]